In [4]:
#import libs
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import numpy as np

#####download and unzip dataset

In [ ]:
#1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT   #validation file
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT" -O car_img-val && rm -rf /tmp/cookies.txt

In [ ]:
!unzip "/content/car_img-val" -d "/content/"

In [5]:
allfiles = os.listdir('/content/validation')
print(len(allfiles))

4240


In [6]:
!rm -rf sample_data
!rm -rf car_img-val

In [7]:
path = '/content/'
source = '/content/validation'
destination = '/content/train'
os.rename(source, destination)

##### define some functions for convert .xml label files to yolo format

In [9]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
              
        # Get details of the bounding box 
        if elem.tag == "object":
          bbox = {}
          for subelem in elem:
              if subelem.tag == "name":
                  bbox["class"] = subelem.text
                    
              elif subelem.tag == "bndbox":
                  for subsubelem in subelem:
                       bbox[subsubelem.tag] = int(float(subsubelem.text))           
          info_dict['bboxes'].append(bbox)
    
    return info_dict


# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"کل ناحیه پلاک": 0}


# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov7(info_dict, ann, img):

  print_buffer = []
    
    # For each bounding box
  for b in info_dict["bboxes"]:
    if b['class'] == 'کل ناحیه پلاک':
      try:
          class_id = class_name_to_id_mapping[b["class"]]
      except KeyError:
          print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
          
          # Transform the bbox co-ordinates as per the format required by YOLO v7
      b_center_x = (b["xmin"] + b["xmax"]) / 2 
      b_center_y = (b["ymin"] + b["ymax"]) / 2
      b_width    = (b["xmax"] - b["xmin"])
      b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
      #img = ann[:-3] + 'jpg'
      im=plt.imread(img)
      image_h, image_w, image_c = im.shape
      b_center_x /= image_w 
      b_center_y /= image_h 
      b_width    /= image_w 
      b_height   /= image_h 
        
        #Write the bbox details to the file 
      print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
  save_file_name = ann[:-3] + 'txt'
  print("\n".join(print_buffer), file= open(save_file_name, "w")) 

def movtofoldertrain(path):
  src = path + 'train'
  dest1 = path + 'train/images'
  dest2 = path + 'train/labels'
  allfiles = os.listdir(src)
  for f in allfiles:
    print(f)
    if f.endswith(".jpg"):
      src_path = os.path.join(src, f)
      dst_path = os.path.join(dest1, f)
      os.rename(src_path, dst_path)
    elif f.endswith(".txt"):
      src_path = os.path.join(src, f)
      dst_path = os.path.join(dest2, f)
      os.rename(src_path, dst_path)
    elif f.endswith(".xml"):
      src_path = os.path.join(src, f)
      os.remove(src_path)
    else:
      pass

In [ ]:
searchxml   = os.path.join( path , "*" , "*.xml" )
xmlfiles = sorted(glob.glob( searchxml ))
i=0
for xml in xmlfiles:
  try:
    ann = xml
    img = ann[:-3] + 'jpg'
    print(i,xml)
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov7(info_dict, ann, img)
    i = i + 1
  except:
    print(xml)
print(".txt ok")

os.mkdir('test')
Paths=['train/images', 'train/labels', 'test/images', 'test/labels']
for p in Paths:
  os.mkdir(path+p)

movtofoldertrain(path)
print("movtofoldertrain ok")

In [11]:
jpgfiles = sorted(os.listdir('/content/train/images'))
txtfiles = sorted(os.listdir('/content/train/labels'))
srcf = path + 'train/images'
srcg = path + 'train/labels'

dest1 = path + 'test/images'
dest2 = path + 'test/labels'
for i in range (0,2119,5):

  f=jpgfiles[i]
  g=txtfiles[i]

  src_pathf = os.path.join(srcf, f)
  dst_pathf = os.path.join(dest1, f)
  os.rename(src_pathf, dst_pathf)

  src_pathg = os.path.join(srcg, g)
  dst_pathg = os.path.join(dest2, g)
  os.rename(src_pathg, dst_pathg)
  
print("movtofoldertest ok")

movtofoldertest ok


In [12]:
trainjpg = sorted(os.listdir('/content/train/images'))
traintxt = sorted(os.listdir('/content/train/labels'))
testjpg = sorted(os.listdir('/content/test/images'))
testtxt = sorted(os.listdir('/content/test/labels'))

print(len(trainjpg), len(traintxt), len(testjpg), len(testtxt)) #1696 1696 424 424

1696 1696 424 424


#####clone repo and train

In [ ]:
#clone repo
!git clone https://github.com/HamidrezaHayatjou/LPRwithyolo7.git

In [14]:
cd LPRwithyolo7

/content/LPRwithyolo7


In [ ]:
#download .pt file
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
#install requirements
!pip install -r /content/LPRwithyolo7/requirements.txt
!pip install -r /content/LPRwithyolo7/requirements_gpu.txt

In [ ]:
#train
!python train.py --workers 8 --device 0 --batch-size 16 --epochs 50 --data data/coco.yaml --img 640 640 --cfg cfg/training/yolov7.yaml --weights 'yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

In [ ]:
#inference
!python detect.py --weights /content/TSDwithyolo7/runs/train/yolov7-custom/weights/best.pt --conf 0.5 --img-size 640 --source /content/LPRwithyolo7/testvideo.mp4 --no-trace